In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os

### Выбор метрики

Так как наша модель обчается принимать решения о болезнях людей, то важнее минимизировать ошибку  false positive - решение, что человек не болен, когда он болен на самом деле. Следовательно.....

## Preprosessing

In [2]:
raw_path = '../data/raw/'
train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [3]:
from sklearn.model_selection import train_test_split

TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
train, target = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]
X_train, X_val, y_train, y_val = train_test_split(train, target)